#####   Getting and preprocessing input data

In [1]:
# Set working directory and path to shapefile of study area

working_dir = ''  #provide working folder directory
study_area = ''  #provide path to shapefile of study_area
train_start_date = ''  # provide start date in format e.g. '2016-12-31'
train_end_date = ''  #provide end date in format e.g. '2016-12-31'
val_start_date = ''
val_end_date = ''

In [2]:
from bakaano.tree_cover import VCF
vf = VCF(working_dir, study_area)
vf.download_vcf()

In [ ]:
# Get elevation data

from bakaano.dem import DEM
dd = DEM(working_dir, study_area, local_data=False, local_data_path=None)
dd.get_dem_data()
dd.plot_dem()

In [ ]:
# Get soil data

from bakaano.soil import Soil
sgd = Soil(working_dir, study_area)
sgd.get_soil_data()
sgd.plot_soil()

In [ ]:
# Get meteo data

from bakaano.meteo import Meteo
cd = Meteo(working_dir, study_area, local_data=False, local_prep_path=None, local_tasmax_path=None, 
           local_tasmin_path=None, local_tmean_path=None)
prep, tasmax, tasmin, tmean = cd.get_meteo_data()

#####   Training Bakaano-Hydro model 

In [ ]:
#TRAINING THE NEURAL NETWORK MODEL
# Initialize  instance of Bakaano-Hydro model

from bakaano.main import BakaanoHydro
bk = BakaanoHydro(
    working_dir,
    study_area,
    train_start_date,
    train_end_date
    )

In [ ]:
# compute surface runoff and route it to river network

bk.compute_runoff_route_flow(prep, tasmax, tasmin, tmean)


In [ ]:
# Train Bakaano-Hydro for streamflow prediction 

grdc_netcdf = ''   #provide path to netcdf format of observed streamflow obtained from GRDC
bk.train_streamflow_model(grdc_netcdf)

#####   Evaluating Bakaano-Hydro model performance

In [ ]:
#EVALUATING THE NEURAL NETWORK MODEL
from bakaano.main import BakaanoAI
bk = BakaanoAI(
    working_dir,
    study_area,
    val_start_date,
    val_end_date
)


In [ ]:
# compute surface runoff and route it to river network

bk.compute_runoff_route_flow(prep, tasmax, tasmin, tmean)

In [ ]:
# Train deepSTRMM neural network model for streamflow prediction

model_path = ''  #path to trained model. Normally can be found in working_dir/models/
bk.evaluate_streamflow_model(model_path, grdc_netcdf)